# Предварительная обработка текста на Python

In [59]:
'''
%pip install pymorphy2
%pip install pandas
%pip install seaborn
%pip install nltk
'''

'\n%pip install pymorphy2\n%pip install pandas\n%pip install seaborn\n%pip install nltk\n'

Загружаем библиотеки

In [60]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 200
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)

#import math as mth
#import matplotlib.pyplot as plt
#from scipy import stats as st
#from statsmodels.stats.weightstats import ttest_ind
#from scipy.stats import ttest_ind_from_stats

#import numpy as np
#import seaborn as sns; sns.set()
#cm = sns.light_palette("green", as_cmap=True)

import nltk
import pymorphy2
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
#import os
#import io
#import re


[nltk_data] Downloading package punkt to /Users/sna/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Загружаем датасет

In [61]:
#data = pd.read_csv('/Users/sna/Dropbox/git/banks/datasets/banks.csv', sep='\t', index_col='idx', on_bad_lines='skip')



try:
    data = pd.read_csv('/Users/sna/Dropbox/git/banks/datasets/banks.csv', sep='\t', index_col='idx', on_bad_lines='skip')
except:
    data = pd.read_csv(r'C:\Users\skd23\Dropbox\git\banks\datasets\banks.csv', sep='\t', index_col='idx', on_bad_lines='skip')


'''
path = os.path.join('/' + os.sep, 'Users', 'sna', 'Dropbox', 'git', 'banks', 'datasets', 'banks.csv')
data = pd.read_csv(path, sep='\t')
'''
#data = pd.read_csv('/sna/Dropbox/git/banks/datasets/banks.csv')

#path = './datasets/banks.csv'

'''
ROOT = os.path.abspath("/Users/sna/Dropbox/git/banks/datasets/")
directories = os.listdir(ROOT)
for dir in directories:
    print(dir)
'''


'\nROOT = os.path.abspath("/Users/sna/Dropbox/git/banks/datasets/")\ndirectories = os.listdir(ROOT)\nfor dir in directories:\n    print(dir)\n'

Изучим датасет

In [62]:
list_df = (data,)
for list_df in list_df:
    print('\033[31m' + '______________info_____________________' + '\033[0m')
    display(list_df.info())
    print('\033[31m' + '______________describe_________________' + '\033[0m')
    display(list_df.describe().T)
    print('\033[31m' + '______________head_____________________' + '\033[0m')
    display(list_df.head(10))
    print('\033[31m' + '______________sample___________________' + '\033[0m')
    display(list_df.sample(10))
    print('\033[31m' + '______________tail_____________________' + '\033[0m')
    display(list_df.tail(10))
    print('\033[31m' + '_____________isna______________________' + '\033[0m')
    display(list_df.isna().sum().reset_index())
    print('\033[31m' + '_____________duplicated________________' + '\033[0m')
    display(list_df.duplicated().sum())
    print('\033[31m' + '__количество пропущенных значений по каждому из столбцов__' + '\033[0m')
    display(list_df.isnull().mean().reset_index())


______________info_____________________
<class 'pandas.core.frame.DataFrame'>
Int64Index: 13999 entries, 0 to 13998
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Score   13999 non-null  object
 1   Text    13999 non-null  object
dtypes: object(2)
memory usage: 328.1+ KB


None

______________describe_________________


,count,unique,top,freq
Score,13999,2,Positive,7000
Text,13999,13998,"В свое время коллега оставил мой телефон как контактное лицо при получении кредита.Что-то с кредитом не заладилось и теперь каждый раз, когда у него просрочка и не удается дозвониться до него — он...",2


______________head_____________________


...

,index,0
0,Score,0
1,Text,0


_____________duplicated________________


1

__количество пропущенных значений по каждому из столбцов__


,index,0
0,Score,0.0
1,Text,0.0


In [63]:
data.columns

Index(['Score', 'Text'], dtype='object')

Переведем `Text` в нижний регистр

In [64]:
data['Text'] = data['Text'].str.lower()

In [65]:
data['Text']=data['Text'].apply(str)

Определим количество негативных и позитивных оценок

In [66]:
count_score= data.groupby(['Score']).agg({'Text':'count'})\
                                    .sort_values(ascending=False, by='Score').reset_index()
count_score.columns = ['Оценка', 'Количество']
count_score



,Оценка,Количество
0,Positive,7000
1,Negative,6999


Выберем текст для токенизации

In [67]:
text = data.iloc[0]['Text']

In [68]:
text

'в альфа-банке работает замечательная девушка - ильясова орна, вежливая, отзывчивая, действительно участвует в запросе клиента, я приходила к ней подряд ровно три дня, каждый день она помнила время моего прихода, помогла оформить кредит в размере 1млн рублей, когда я пришла с партнером (передавать ей полученный кредит за покупаемый мною авто), специалист ильясова орна помогла нам вывести всю сумму в один день (а это было непросто), так что сделка состоялась и все остались довольны! моя знакомая в конце всего добавила: теперь я \xa0поняла, почему мы пришли в это отделение, к этой девушке..побольше бы таких замечательных специалистов! приобретать программу здоровье, и вообще все, что связано с альфа-банком, теперь буду только у нее!'

In [69]:
#tokens = text.astype(str).apply(word_tokenize)

In [70]:
tokens = word_tokenize(text)


In [71]:
tokens[:10]

['в',
 'альфа-банке',
 'работает',
 'замечательная',
 'девушка',
 '-',
 'ильясова',
 'орна',
 ',',
 'вежливая']

Удалим знаки препинания

In [72]:
punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...']

In [73]:
only_words = []
for token in tokens:
    if token not in punctuation_marks:
        only_words.append(token)

In [74]:
only_words[:10]

['в',
 'альфа-банке',
 'работает',
 'замечательная',
 'девушка',
 'ильясова',
 'орна',
 'вежливая',
 'отзывчивая',
 'действительно']

Приводим слова к нормальной форме

In [75]:
morph = pymorphy2.MorphAnalyzer()

In [76]:
lemmas = []
for token in only_words:
    lemmas.append(morph.parse(token)[0].normal_form)

In [77]:
lemmas[:10]

['в',
 'альфа-банк',
 'работать',
 'замечательный',
 'девушка',
 'ильясова',
 'орный',
 'вежливый',
 'отзывчивый',
 'действительно']

удалим стоп-слова

In [78]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/sna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [79]:
stop_words = stopwords.words('russian')

In [80]:
stop_words[:10]

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

Функция предварительной обработки

In [81]:
def preprocess(text, stop_words, punctuation_marks, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return preprocessed_text

In [82]:
punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...']
stop_words = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()

In [83]:
#data[0:10].apply(lambda row: preprocess(row['Text'], punctuation_marks, stop_words, morph), axis=1)

In [84]:
data['Preprocessed_texts'] = data.apply(lambda row: preprocess(row['Text'], punctuation_marks, stop_words, morph), axis=1)

In [85]:
data

,Score,Text,Preprocessed_texts
idx,,,
0,Positive,"в альфа-банке работает замечательная девушка - ильясова орна, вежливая, отзывчивая, действительно участвует в запросе клиента, я приходила к ней подряд ровно три дня, каждый день она помнила время...","[альфа-банк, работать, замечательный, девушка, ильясова, орный, вежливый, отзывчивый, действительно, участвовать, запрос, клиент, приходить, подряд, ровно, день, каждый, день, помнить, время, мой,..."
1,Negative,"оформляя рассрочку в м. видео в меге тёплый стан. попросила менеджера, чтобы заявка была без страховки. в итоге, когда подписывала договор, в договоре оказались лишние почти 6 тысяч, когда менедже...","[оформлять, рассрочка, м., видео, мег, тёплый, стан, попросить, менеджер, заявка, страховка, итог, подписывать, договор, договор, оказаться, лишний, 6, тысяча, менеджер, стать, спрашивать, это, 6,..."
2,Positive,"очень порадовала оперативность работы в банке. заказал через интернет карту visa classic, потребовалось подойти в банк для подписания заявления. в тот же день мне открыли текущий счет с доступом ч...","[очень, порадовать, оперативность, работа, банк, заказать, интернет, карта, visa, classic, потребоваться, подойти, банк, подписание, заявление, день, открыть, текущий, счёт, доступ, интернет, услу..."
3,Negative,"имела неосторожность оформить потреб. кредит в альфа-банке - очень жалею. во-первых, карту при оформлении кредита мне не выдали (объяснили тем, что их сейчас нет в наличии). во-вторых, никакую инф...","[иметь, неосторожность, оформить, потреба, кредит, альфа-банк, очень, жалеть, во-первых, карта, оформление, кредит, выдать, объяснить, наличие, во-вторых, никакой, информация, телефон, предоставля..."
4,Negative,"небольшая предыстория: нашел на сайте mdm банка интересную ко-бренд карту «техносила – mdm банк», подал заявку на нее, и скоро позвонили с горячей линии. предложили кредитную карту, на что я в при...","[небольшой, предыстория, найти, сайт, mdm, банк, интересный, ко-бренд, карта, «, техносить, –, mdm, банк, », подать, заявка, скоро, позвонить, горячий, линия, предложить, кредитный, карта, принцип..."
...,...,...,...
13994,Positive,"о высокой надёжности мкб, порядочности и добросовестности его сотрудников. кратко: 19.6.12 г., открывая счёт, я, по рассеянности внёс в кассу сумму на 25000 р. больше величины счёта. переплату обн...","[высокий, надёжность, мкб, порядочность, добросовестность, сотрудник, кратко, 19.6.12, г., открывать, счёт, рассеянность, внести, касса, сумма, 25000, р., величина, счёт, переплата, обнаружить, ли..."
13995,Positive,"обслуживаюсь в офисе на чернореченской 42а, как физ. лицо уже год. в офисе на самом деле никогда нет очереди, всегда есть свободный сотрудник, который готов помочь в любом вопросе. при дистанционн...","[обслуживаться, офис, чернореченский, 42а, физ, лицо, год, офис, сам, дело, очередь, свободный, сотрудник, который, готовый, помочь, любой, вопрос, дистанционный, обслуживание, трудность, никакой,..."
13996,Positive,"попала сегодня в очень неприятную ситуацию. решила погасить долг по карте. пришла в банкомат (пр. дзержинского, 1/4/). банкомат деньги принял, выдал чек о зачислении, вернул карту, а напоследок пр...","[попасть, сегодня, очень, неприятный, ситуация, решить, погасить, долг, карта, прийти, банкомат, пр, дзержинский, 1/4/, банкомат, деньга, принять, выдать, чек, зачисление, вернуть, карта, напослед..."


In [86]:
data.columns

Index(['Score', 'Text', 'Preprocessed_texts'], dtype='object')

In [87]:
prep_text = data['Preprocessed_texts']

In [88]:
prep_text.to_frame()

,Preprocessed_texts
idx,
0,"[альфа-банк, работать, замечательный, девушка, ильясова, орный, вежливый, отзывчивый, действительно, участвовать, запрос, клиент, приходить, подряд, ровно, день, каждый, день, помнить, время, мой,..."
1,"[оформлять, рассрочка, м., видео, мег, тёплый, стан, попросить, менеджер, заявка, страховка, итог, подписывать, договор, договор, оказаться, лишний, 6, тысяча, менеджер, стать, спрашивать, это, 6,..."
2,"[очень, порадовать, оперативность, работа, банк, заказать, интернет, карта, visa, classic, потребоваться, подойти, банк, подписание, заявление, день, открыть, текущий, счёт, доступ, интернет, услу..."
3,"[иметь, неосторожность, оформить, потреба, кредит, альфа-банк, очень, жалеть, во-первых, карта, оформление, кредит, выдать, объяснить, наличие, во-вторых, никакой, информация, телефон, предоставля..."
4,"[небольшой, предыстория, найти, сайт, mdm, банк, интересный, ко-бренд, карта, «, техносить, –, mdm, банк, », подать, заявка, скоро, позвонить, горячий, линия, предложить, кредитный, карта, принцип..."
...,...
13994,"[высокий, надёжность, мкб, порядочность, добросовестность, сотрудник, кратко, 19.6.12, г., открывать, счёт, рассеянность, внести, касса, сумма, 25000, р., величина, счёт, переплата, обнаружить, ли..."
13995,"[обслуживаться, офис, чернореченский, 42а, физ, лицо, год, офис, сам, дело, очередь, свободный, сотрудник, который, готовый, помочь, любой, вопрос, дистанционный, обслуживание, трудность, никакой,..."
13996,"[попасть, сегодня, очень, неприятный, ситуация, решить, погасить, долг, карта, прийти, банкомат, пр, дзержинский, 1/4/, банкомат, деньга, принять, выдать, чек, зачисление, вернуть, карта, напослед..."


In [89]:
clean_df = data[data['Preprocessed_texts'].str.len() >= 100]

In [90]:
clean_df.Preprocessed_texts.to_frame()

,Preprocessed_texts
idx,
1,"[оформлять, рассрочка, м., видео, мег, тёплый, стан, попросить, менеджер, заявка, страховка, итог, подписывать, договор, договор, оказаться, лишний, 6, тысяча, менеджер, стать, спрашивать, это, 6,..."
2,"[очень, порадовать, оперативность, работа, банк, заказать, интернет, карта, visa, classic, потребоваться, подойти, банк, подписание, заявление, день, открыть, текущий, счёт, доступ, интернет, услу..."
4,"[небольшой, предыстория, найти, сайт, mdm, банк, интересный, ко-бренд, карта, «, техносить, –, mdm, банк, », подать, заявка, скоро, позвонить, горячий, линия, предложить, кредитный, карта, принцип..."
6,"[месяц, назад, взять, автокредит, ходить, разный, салон, форд, присматривать, машина, исходить, конкретный, сумма, автомобиль, конкретный, первоначальный, взнос, автосалон, навязывать, именно, кре..."
8,"[день, добрый, мама, с-на, в.а.,1932г, рождение, пенсионерка, являться, клиент, бинбанка, октябрь, 2015, год, снять, пенсия, карта, срок, карта, закончиться, её, необходимо, перевыпустить, с-ин, г..."
...,...
13991,"[остальной, остаться, начаться, смешно, сотрудник, принимать, заявка, кредит, стать, суть, соучастник, единственный, отзыв, месяц, который, бинбанк, мдм, банк, иметь, двое, практически, сразу, вст..."
13993,"[здравствуйте, мой, знакомство, хоум, кредит, банк, начинаться, начальный, этап, складываться, очень, хороший, впечатление, банк, очень, часто, пользоваться, кредит, основное, это, товарный, креди..."
13996,"[попасть, сегодня, очень, неприятный, ситуация, решить, погасить, долг, карта, прийти, банкомат, пр, дзержинский, 1/4/, банкомат, деньга, принять, выдать, чек, зачисление, вернуть, карта, напослед..."
